In [ ]:
!pip install --upgrade google-cloud-storage pandas google-cloud-bigquery

#Uploading to GCS

In [43]:
from google.cloud import storage
from google.cloud import bigquery
!pip install google-cloud-bigquery
import pandas as pd
import json
import os
import numpy as np

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
project_id = 'evictionsdataset'
bucket_name = 'evictions_data'
json_file_name = 'data.json'

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving Evictions.csv to Evictions (1).csv


In [ ]:
csv_file_path = list(uploaded.keys())[0]

In [ ]:
df = pd.read_csv(csv_file_path)

In [ ]:
csv_file_path = list(uploaded.keys())[0]
print(f"CSV file uploaded: {csv_file_path}")
try:
    df = pd.read_csv(csv_file_path)
    print("CSV file read successfully.")
except Exception as e:
    print(f"Error reading CSV file: {e}")
try:
    json_data = df.to_json(orient='records')
    print("DataFrame converted to JSON successfully.")
except Exception as e:
    print(f"Error converting DataFrame to JSON: {e}")
try:
    with open(json_file_name, 'w') as json_file:
        json.dump(json.loads(json_data), json_file, indent=2)
    print(f"JSON data written to {json_file_name} successfully.")
except Exception as e:
    print(f"Error writing JSON data to file: {e}")
def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    try:
        storage_client = storage.Client(project=project_id)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(source_file_name)
        print(f"File {source_file_name} uploaded to {destination_blob_name}.")
    except Exception as e:
        print(f"Error uploading file to GCS: {e}")

upload_to_gcs(bucket_name, json_file_name, json_file_name)

CSV file uploaded: Evictions (1).csv
CSV file read successfully.
DataFrame converted to JSON successfully.
JSON data written to data.json successfully.
File data.json uploaded to data.json.


In [ ]:
df = pd.read_csv(csv_file_path)

In [ ]:
df.head()

,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Postcode,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,79440/17,16067,114-90 225 STREET,NaN,06/01/2018,Edward,Guida,Residential,QUEENS,11411,Not an Ejectment,Possession,40.699198,-73.732949,13.0,27.0,594.0,4242871.0,4.112680e+09,Cambria Heights
1,17350/18,344660,2334 BOSTON ROAD,14H,06/27/2018,Thomas,Bia,Residential,BRONX,10467,Not an Ejectment,Possession,40.860558,-73.867067,11.0,13.0,330.0,2050046.0,2.043470e+09,Bronxdale
2,46070/17,76991,1771 MONROE AVENUE,1B,11/28/2017,Henry,Daley,Residential,BRONX,10457,Not an Ejectment,Possession,40.846168,-73.906539,5.0,15.0,22901.0,2007451.0,2.027970e+09,Mount Hope
3,63669/16,464171,818 BECK STREET,4-N,01/25/2017,Danny,Weinheim,Residential,BRONX,10459,Not an Ejectment,Possession,40.817572,-73.897436,2.0,17.0,87.0,2005590.0,2.027090e+09,Longwood
4,86267/17,75677,887 FULTON STREET,2L,03/19/2018,Justin,Grossman,Residential,BROOKLYN,11238,Not an Ejectment,Possession,40.683699,-73.967799,2.0,35.0,199.0,3056594.0,3.019760e+09,Clinton Hill


#transformation

In [ ]:
df['Executed Date'] = pd.to_datetime(df['Executed Date'], errors='coerce')
df['Year'] = df['Executed Date'].dt.year
df['Quarter'] = df['Executed Date'].dt.quarter
df['Month'] = df['Executed Date'].dt.month
df['Day'] = df['Executed Date'].dt.day
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
valid_boroughs = ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']
df = df[df['BOROUGH'].isin(valid_boroughs)]
numeric_columns = ['Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL', 'Latitude', 'Longitude']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df['Board_District_Sum'] = df['Community Board'] + df['Council District']

In [ ]:
json_data = df.to_json(orient='records')
json_file_name = 'transformed_data.json'
with open(json_file_name, 'w') as json_file:
    json.dump(json.loads(json_data), json_file, indent=2)

print(f"Transformed data written to {json_file_name} successfully.")

Transformed data written to transformed_data.json successfully.


In [ ]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    try:
        storage_client = storage.Client(project='evictionsdataset')
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_filename(source_file_name)
        print(f"File {source_file_name} uploaded to {destination_blob_name}.")
    except Exception as e:
        print(f"Error uploading file to GCS: {e}")
bucket_name = 'evictions_data'
source_file_name = 'transformed_data.json'
destination_blob_name = 'evdata.json'
upload_to_gcs(bucket_name, source_file_name, destination_blob_name)


File transformed_data.json uploaded to evdata.json.


In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account
import json
key_path = "C:\\Users\\guttman\\Documents\\EV_data\\evictionsdataset-4d5d2dabe640.json"
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)
dataset_id = 'evictionsdata'
table_id = 'Evictions_NYC'
with open('path/to/your/transformed.json', 'r') as f:
    json_data = json.load(f)

rows_to_insert = [json_data]
errors = client.insert_rows_json(
    f"{dataset_id}.{table_id}",
    rows_to_insert,
    row_ids=[None] * len(rows_to_insert),
    schema=schema
)

if errors == []:
    print("New rows have been added.")
else:
    print("Errors occurred: ", errors)
